In [1]:
!pip install ReliefF 
!pip install sklearn_relief
!pip install mrmr_selection

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# New Section

In [2]:
import numpy as np
import pandas as pd
from scipy import stats
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.preprocessing import label_binarize
from sklearn.metrics import accuracy_score

from sklearn import metrics
from sklearn.model_selection import train_test_split
from xgboost import plot_importance
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.multiclass import OneVsRestClassifier
import sklearn_relief as sr
from scipy.io import arff
from functools import reduce
from mrmr import mrmr_classif
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier


In [3]:
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression, mutual_info_classif
from sklearn.feature_selection import SelectKBest,SelectPercentile,f_classif ,r_regression,SelectFdr,RFE
from sklearn import preprocessing

from sklearn.preprocessing import label_binarize
from sklearn.svm import SVR
from sklearn import metrics


In [4]:
filePath = "./SPECTF.train.csv"
n_featurs_selected = 25


In [5]:
X = pd.read_csv(filePath,header = None)
df = pd.read_csv(filePath,header = None)

#clase = the column we would like to classify
y = X.iloc[:,0]
print(X.shape)

#drops = Array of columns drop
X.drop( columns = X.columns[0], axis=1, inplace=True)
print(X.shape)
print(y.shape)
print(X.columns)

(80, 45)
(80, 44)
(80,)
Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
            18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
            35, 36, 37, 38, 39, 40, 41, 42, 43, 44],
           dtype='int64')


The correletion of featurs to the the target by Information-Gain

In [6]:
#information gain
def IF_selection(X,y,n_featurs_selected):
  selector = SelectKBest(mutual_info_classif,k = n_featurs_selected )
  X_reduced = selector.fit_transform(X,y)
  cols = selector.get_support(indices = True)
  selected_columns = X.iloc[:,cols].columns.tolist()
  # mi = mutual_info_classif(X, y)
  # mi = pd.Series(mi)
  # mi.index = X.columns
  # mi_sort = mi.sort_values(ascending=False)
  # best_k =  mi_sort[:n_featurs_selected,]
  # mi.sort_values(ascending=True).plot.bar(figsize=(10, 4))
  # IF_k_best = []
  # for x in best_k.items():
  #   IF_k_best.append(x[0])
  # IF_k_best = np.array(IF_k_best)
  return selected_columns
# IF_k_best = IF_selection(X,y,n_featurs_selected)


Correlation-based Feature Selection, CFS: This is a simple filter algorithm that ranks feature subsets according to a correlation-based heuristic evaluation function [17]. The bias of the evaluation function is toward subsets that contain features that are highly correlated with the class and uncorrelated with each other. Irrelevant features should be ignored and redundant features should be screened out.

In [7]:
import numpy as np
from google.colab import files

from mutual_information import su_calculation

#need to load the folder CFSmethod from the git below
#https://github.com/ZixiaoShen/Correlation-based-Feature-Selection/blob/master/CFSmethod/CFS.py
def merit_calculation(X, y):
    """
    This function calculates the merit of X given class labels y, where
    merits = (k * rcf) / sqrt (k + k*(k-1)*rff)
    rcf = (1/k)*sum(su(fi, y)) for all fi in X
    rff = (1/(k*(k-1)))*sum(su(fi, fj)) for all fi and fj in X
    :param X:  {numpy array}, shape (n_samples, n_features) input data
    :param y:  {numpy array}, shape (n_samples) input class labels
    :return merits: {float}  merit of a feature subset X
    """

    n_samples, n_features = X.shape
    rff = 0
    rcf = 0
    for i in range(n_features):
        fi = X[:, i]
        rcf += su_calculation(fi, y)  # su is the symmetrical uncertainty of fi and y
        for j in range(n_features):
            if j > i:
                fj = X[:, j]
                rff += su_calculation(fi, fj)
    rff *= 2
    merits = rcf / np.sqrt(n_features + rff)
    return merits


def cfs(X, y):
    """
    This function uses a correlation based heuristic to evaluate the worth of features which is called CFS
    :param X: {numpy array}, shape (n_samples, n_features) input data
    :param y: {numpy array}, shape (n_samples) input class labels
    :return F: {numpy array}, index of selected features
    """

    n_samples, n_features = X.shape
    F = []
    M = []  # M stores the merit values
    while True:
        merit = -100000000000
        idx = -1
        for i in range(n_features):
            if i not in F:
                F.append(i)
                # calculate the merit of current selected features
                t = merit_calculation(X[:, F], y)
                if t > merit:
                    merit = t
                    idx = i
                F.pop()
        F.append(idx)
        M.append(merit)
        if len(M) > 5:
            if M[len(M)-1] <=M[len(M)-2]:
                if M[len(M)-2] <= M[len(M)-3]:
                    if M[len(M)-3] <= M[len(M)-4]:
                        if M[len(M)-4] <= M[len(M)-5]:
                            break
    return np.array(F)

In [8]:
def cfs_selection(X,y,k):
# x - data set witout label class
# y - label class 
# return vector of best featurs
  cfs_vectore = cfs(X.to_numpy(),y.to_numpy())
  return cfs_vectore[:k]


Consistency-based Filter: The consistency-based filter evaluates the worth of a subset of features by the level of consistency in the class values when the training instances are projected onto the subset of attributes. The algorithm generates a random subset S from the number of features in every round. If the number of features of S is less than the current best, the data with the features prescribed in S is checked against the inconsistency criterion. If its inconsistency rate is below a pre-specified one, S becomes the new current best.






In [9]:
def consistency_feature_selection(X,y,k):
# x - data set witout label class
# y - label class 
# return vector of best featurs

  # K_best = SelectPercentile().fit(X,y)
  # #Consistency-based Filter: The consistency-based filter
  # # Get columns to keep and create new dataframe with those only
  # cols = K_best.get_support(indices=True)
  # features_df_new = X.iloc[:,cols]
  # features_df_new.columns
  # const_k_best = features_df_new.columns.values
  estimator = SVR(kernel="linear")
  selector = RFE(estimator, n_features_to_select=k, step=1)
  selector = selector.fit(X, y)
  names = X.columns[selector.get_support()]
  return names


INTERACT: The INTERACT algorithm  is based on symmetrical uncertainty (SU). Besides SU, INTERACT also
includes the consistency contribution (c-contribution). The
algorithm consists of ranking the features in descending order
based on their SU values and then evaluating one by one
starting from the end of the ranked feature list. Then, a feature
is selected or not depending on a threshold, either established
by the user or employing the default value provided. Theoretically, INTERACT can handle feature interaction

In [10]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier

def interact(X,y,K):
  knn = KNeighborsClassifier(n_neighbors=3)
  sfs = SequentialFeatureSelector(knn, n_features_to_select=K)
  sfs.fit(X, y)
  SequentialFeatureSelector(estimator=KNeighborsClassifier(n_neighbors=3),
                            n_features_to_select=K)
  sfs.get_support()
  cols = sfs.get_support(indices = True)
  selected_columns = X.iloc[:,cols].columns.tolist() 
  return selected_columns


Correlation-based Feature Selection 
Selecting highly correlated features

In [11]:
#Correlation-based Feature Selection
def cor_k_best(df,colum_target,min_cor):
# input :  df - data frame
# colum_target - index colum of target class
# min_cor - minmum corllation
# return : vecotr of best feature
  cor = df.corr()
  cor_target = abs(cor[colum_target])

  relevant_features = cor_target[cor_target>min_cor]
  cor_k_best = []
  for x in relevant_features.items():
    if(x[0] == colum_target ):
      continue
    cor_k_best.append(x[0])
  cor_k_best_feature = np.array(cor_k_best)

  return cor_k_best_feature

# cor_k_best_feature = cor_k_best(df,0,0.3)
# cor_k_best_feature

ReliefF  is an extension of the original Relief algorithm  that adds the ability of dealing with multiclass problems and it is more robust and capable of dealing with incomplete and noisy data. The Relief family of methods are specially attractive because they may be applied in all situations, have low bias, include interaction among features and may capture local dependencies that other methods miss.

In [12]:
import sklearn_relief as relief

def relif_k_best(X,y,n_featurs_selected):
# 
# x - data set witout label class
# y - label class 
# n_featurs_selected - number of k best featurs
# return vector size of n_featurs_selected best features
# 
  my_input_matrix = np.array(X)
  my_label_vector = np.array(y)
  r = relief.Relief(
      n_features=X.shape[1] # Choose the best K features
  ) # Will run by default on all processors concurrently

  my_transformed_matrix = r.fit_transform(
      my_input_matrix,
      my_label_vector
  )
  dp_rate_index = pd.DataFrame(r.w_, index = None)
  dp_rate_index.head()
  dp_rate_index_sort = dp_rate_index.sort_values([0],axis = 0, ascending= False)
  relief_k_best = []
  scores = []
  for x in dp_rate_index_sort.iterrows():
    relief_k_best.append(x[0])
    scores.append(x[1][0])
  relief_k_best = relief_k_best[:n_featurs_selected]
  scores = scores[:n_featurs_selected]
  return relief_k_best,scores

relief_k_best, scores = relif_k_best(X,y,n_featurs_selected)



all the featur that selected from all our 5 five differents feature selection


The complete selection algo

In [13]:
def Ensemble_selection(data,min_cor,colum_target,K = 25):
  #return vector of 0,1 for selecting index of fetur colum

  y = data.iloc[:,colum_target]
  X = data.drop( columns = data.columns[colum_target], axis=1, inplace=False)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  IF_k_best = IF_selection(X,y,K)
  IF_k_best = [ data.columns.get_loc(x) for x in IF_k_best ]
  cfs_vectore = cfs_selection(X,y,K)

  const_k_best = consistency_feature_selection(X,y,K)
  const_k_best = [ data.columns.get_loc(x) for x in const_k_best ]

  # interact_k_best = interact(X,y,n_featurs_selected)
  # print(interact_k_best)
  # cor_k_best_featur = cor_k_best(data,colum_target,min_cor)
  # print(cor_k_best_featur)

  relief_k_best = relif_k_best(X,y,K)
  relief_k_best = relief_k_best[0]
  feature_selection = []
  feature_selection =reduce(np.union1d, (relief_k_best,const_k_best,cfs_vectore,IF_k_best))

  zeros = np.zeros((X.shape[1]+1,), dtype=int)
  # for x in np.nditer(feature_selection):
  #   zeros[x] = 1
  return zeros,feature_selection

# featurs = Ensemble_selection(df,0.3,0)



In [14]:
featurs = []
print("the number in the vectors represent the featur index in the dataset ")
for i in range(3):
  featurs.append(Ensemble_selection(df,0.3,0)[1])
  print("Index with 1 mean selected feature: ")
  print(featurs[i])
  print(" ")


the number in the vectors represent the featur index in the dataset 
Index with 1 mean selected feature: 
[ 0  1  2  3  4  5  7  8  9 10 11 12 13 14 16 17 19 21 22 23 24 25 26 27
 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44]
 
Index with 1 mean selected feature: 
[ 0  1  2  3  4  5  7  8  9 10 11 12 14 16 17 18 19 21 22 23 24 25 26 27
 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44]
 
Index with 1 mean selected feature: 
[ 0  1  2  3  4  5  7  8  9 10 11 12 13 14 16 17 19 21 22 23 24 25 26 27
 28 29 30 31 32 33 34 35 37 38 39 40 41 42 43 44]
 


In [15]:
import os
df.to_csv('FINAL.csv')

In [16]:
# file1 file2 from bioconductor folder
#
file1 = './CLL.csv'
file2 = './ayeastCC.csv'
 
file1_pd = pd.read_csv(file1,header = None, index_col = 0)
file1_pd = file1_pd.T
file1_pd.drop(columns=file1_pd.columns[0], axis=1, inplace=True)
first_column = file1_pd.pop('Class')
file1_pd.insert(file1_pd.shape[1], 'Class', first_column)

file2_pd = pd.read_csv(file2,header = None, index_col = 0)
file2_pd = file2_pd.T
file2_pd.drop(columns=file2_pd.columns[0], axis=1, inplace=True)
first_column = file2_pd.pop('Class')
file2_pd.insert(file2_pd.shape[1], 'Class', first_column)







In [17]:
# from folder ARFF got Lymphoma.arff and Breast.arff
data_breast = arff.loadarff("./Breast.arff")
Breast_df = pd.DataFrame(data_breast[0])



In [18]:
Ovarian = arff.loadarff("./Ovarian.arff")
Ovarian = pd.DataFrame(Ovarian[0])
Ovarian.shape

(253, 15155)

Check for NA in taget class

In [19]:
df_arr  = [file1_pd,file2_pd,Breast_df,Ovarian]


Preprocessing the data


In [20]:
'''
This function is responsible for reading the data from the csv file, data Pre-processin,
Converting Categorical Variables into numerical, splitting the data into train and test sets and returning them
fitting the modle and returning its hyper parameters
'''
from sklearn.feature_selection import VarianceThreshold
import scipy
from scipy.sparse import csr_matrix

def classify2(df):
  # categorial missing values fill with the most common one
  # numerical misssing values fill with the mean.
  # cf = scipy.sparse.csr_matrix(df.values)
  # selector = VarianceThreshold()
  # selector.fit_transform(cf)
  # df = pd.DataFrame.sparse.from_spmatrix(cf)

  for col in df :
    if df[col].dtype == np.object: 
      df[col].fillna(df[col].mode()[0], inplace = True)
    else :
      df[col].fillna(df[col].mean(),inplace = True)

  #Converting Categorical Variables into numerical
  label_encoder = preprocessing.LabelEncoder()
  for col in df.columns:
    if df[col].dtype == np.object:
      df[col] = label_encoder.fit_transform(df[col])

  #clase = the column we would like to classify

  
  return df



In [21]:
# X_y_arr = [(X,y of file 1), (X,y of file2) , (X,y of file 3), (X,y of file 4)]
X_y_arr = [ (classify2(x)) for x in df_arr]

In [22]:
# selected_features = mrmr_classif(X=X, y=y, K=10, return_scores=True)
def mrmr_scores_names(selected_features):
  scores = []
  feature_name = []
  for x in selected_features[1][selected_features[0]].items():
    scores.append(x[1]) #scors
    feature_name.append(x[0]) # return names
    index_col =X.columns.get_loc(x[0]) #col index
  return feature_name,scores,index_col


In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import average_precision_score, precision_recall_curve , auc
from sklearn.metrics import matthews_corrcoef
import time

K = [1,2,3,4,5,10,15,20,25,30,50,100]
models = [('LR',LogisticRegression()), ('RF',RandomForestClassifier()),('Naive_Bayes',GaussianNB()),('KNN',KNeighborsClassifier(n_neighbors=3))]
ft_selection = [('FDR',SelectFdr(alpha = 0.1)), ('RFE', RFE(SVC())), ('Relief', relif_k_best),('mRMR',mrmr_classif),("MyAlgo-Ensamble",Ensemble_selection)]
datasets_name_mapper = {0 : 'CLL', 1 : 'ayeastCC', 2 : 'Breast', 3 : 'Ovarian'}
cv_type = {50 : 'LeavePairOut', 100 : 'LeaveOneOut', 200: 'Fold CV10', 1000 : 'Folds CV5'}
results = { 'Dataset Name': [],'Number of samples': [],'Original Number of features' : [], 'Filtering Algorithm': [],'Learning algorithm': [],
           'Number of features selected':[], 'CV Method & FoldNumer' : [], 'AUC': [], 'ACC':[] ,'List of Selected Features Names (Long STRING)':[],'Selected Features scores': [],'Selection_time_sec': [], 'Fit_time_sec': [],'Inference time':[]}

for idx, dataset in enumerate(X_y_arr[1:]):
  print(datasets_name_mapper[idx])
  for name, model in models:
    print(name,model)
    for ft, ft_select in ft_selection:
      print(ft)
      for k in K:
        print(k)
        X, y = dataset.drop('Class', axis = 1), dataset['Class']
        origin_N_featurs = X.shape[1]
        if ft == 'RFE':
          estimator = SVR(kernel="linear")
          selector = RFE(estimator, n_features_to_select=k, step=1)
          start = time.time()
          new_X = selector.fit_transform(X, y)
          stop = time.time()
          time_select = stop - start
          scores = []
          names = []
          for x in selector.get_feature_names_out():
            name = X.columns.get_loc(x)
            scores.append(selector.ranking_[name])
            names.append(x)
        elif ft == 'Relief':
          cols,scores = ft_select(X, y, k)
          names = X.columns[cols]
          new_X = X.iloc[:,cols]
        elif ft == 'mRMR':
          cols = mrmr_classif(X=X, y=y, K=k,return_scores=True)
          names,scores,cols_index = mrmr_scores_names(cols)
          new_X = X.iloc[:,cols_index]
        elif ft == 'MyAlgo-Ensamble':
          start = time.time()
          t = dataset.columns.get_loc('Class')
          featurs = Ensemble_selection(dataset,0.3,t,k)[1]
          stop = time.time()
          time_select = stop - start
          names = featurs
          scores = []
         
        else:
          start = time.time()
          new_X = ft_select.fit_transform(X, y)
          stop = time.time()
          time_select = stop - start
          names = []
          scores = []
        N = new_X.shape[0]
        if N < 50:
          cv = N-2
          t = 50
        if N >= 50 and  N <= 100:
          cv = N-1
          t = 100
        if N >100 and N < 1000 :
          cv = 10
          t= 200
        if N > 1000:
          cv = 5
          t = 1000
        try:
          res = cross_validate(model,new_X, y, scoring=('accuracy', 'roc_auc','recall'), cv = cv)
        except:
          res = cross_validate(model,new_X, y, scoring=('accuracy', 'roc_auc','recall'), cv = 10)
        stop = time.time()
        fit_model_time = stop - start
        results['Dataset Name'].append(datasets_name_mapper[idx])
        results['Number of samples'].append(N)
        results['Original Number of features'].append(origin_N_featurs)
        results['Filtering Algorithm'].append(ft)
        results['Learning algorithm'].append(name)
        results['Number of features selected'].append(k)
        results['CV Method & FoldNumer'].append(cv_type[t])
        results['AUC'].append(res['test_roc_auc'].mean())
        results['ACC'].append(res['test_accuracy'].mean())
        results['List of Selected Features Names (Long STRING)'].append(names)
        results['Selected Features scores'].append(scores)
        results['Inference time'].append(fit_model_time)
        results['Selection_time_sec'].append(time_select)
        results['Fit_time_sec'].append(fit_model_time)





In [ ]:
import pandas as pd
new_Df = pd.DataFrame(results)
new_Df.to_csv('./result')

In [ ]:
import sklearn
sklearn.metrics.SCORERS.keys()